In [414]:
import json
import cv2
import os
import albumentations as A

In [415]:
def display(*img):
    for i,im in enumerate(img):
        cv2.imshow(str(i),im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [416]:
save_dir='aug_images'
main_path='main_data'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
def get_images():
    images=os.listdir(main_path+'/main_images')
    temp={}
    for i in images:
        temp[i]=cv2.imread(os.path.join(main_path+'/main_images',i)).copy()
        cv2.imwrite(os.path.join(save_dir,i),temp[i])
    return temp
get_images()

{'1.jpeg': array([[[241, 234, 231],
         [241, 234, 231],
         [241, 234, 231],
         ...,
         [118, 126, 116],
         [118, 126, 116],
         [118, 126, 116]],
 
        [[241, 234, 231],
         [241, 234, 231],
         [241, 234, 231],
         ...,
         [118, 126, 116],
         [118, 126, 116],
         [118, 126, 116]],
 
        [[241, 234, 231],
         [241, 234, 231],
         [241, 234, 231],
         ...,
         [118, 126, 116],
         [118, 126, 116],
         [118, 126, 116]],
 
        ...,
 
        [[194, 196, 204],
         [193, 195, 203],
         [192, 194, 202],
         ...,
         [197, 200, 204],
         [196, 199, 203],
         [196, 199, 203]],
 
        [[194, 196, 204],
         [193, 195, 203],
         [192, 194, 202],
         ...,
         [197, 200, 204],
         [195, 198, 202],
         [193, 196, 200]],
 
        [[195, 197, 205],
         [194, 196, 204],
         [192, 194, 202],
         ...,
         [198, 201

In [417]:
Main_data=open(os.path.join(main_path,'main_json.json')).read()
Main_data=json.loads(Main_data)
type(Main_data)

dict

In [418]:
def get_depth(d):

    o = []
    def v_check(key,val):

        if key in filter or val in filter :
            o.append(str(f'{key}:{val}'))

    def check(d):
        if isinstance(d, dict):

            for k,v in d.items():

                if isinstance(v, (dict, list)):
                    check(v)

                else:
                    v=str(v)
                    if  v!='False':
                        v_check(k,v)
                   

        elif isinstance(d, (list)):
            for v in d:

                if isinstance(v, (dict, list)):

                    check(v)

                else:
                    pass
                    # v_check(v)

    check(d)
    return o

data=get_depth(Main_data)
# data

In [419]:
def augment_img(filename, image, multiple_points):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    bboxes = multiple_points
    transform = A.Compose(
        [
            A.Resize(width=1920, height=1080),
            A.RandomCrop(width=1280, height=720),
            A.Rotate(limit=40, p=0.9, border_mode=cv2.BORDER_CONSTANT),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.1),
            # A.RGBShift(r_shift_limit=25, g_shift_limit=25, b_shift_limit=25, p=0.9),
            A.OneOf([
                A.Blur(blur_limit=3, p=0.5),
                # A.ColorJitter(p=0.5),
            ], p=1.0),
        ], bbox_params=A.BboxParams(format="pascal_voc", min_area=2048,
                                    min_visibility=0.3, label_fields=[])
    )

    for i in range(10):
        augmentations = transform(image=image, bboxes=bboxes)
        augmented_img = augmentations["image"]

        if len(augmentations["bboxes"]) == 0:
            continue
        augmented_img = cv2.cvtColor(augmented_img, cv2.COLOR_RGB2BGR)
        for j in augmentations["bboxes"]:
            # print(i)
            x1, y1, x2, y2, typ = map(int, j)
            # cv2.rectangle(augmented_img,(x1,y1),(x2,y2),255,2)
            # display(augmented_img)
            f_name = f'aug_{filename}_{i}.jpg'
            cv2.imwrite(save_dir+'/'+f_name, augmented_img)
            file_key = f_name+str(os.path.getsize(save_dir+'/'+f_name))
            if file_key in Main_data:
                Main_data[file_key]['regions'].append(
                    {'shape_attributes': {'name': 'rect',
                                          'x': x1,
                                          'y': y1,
                                          'width': x2-x1,
                                          'height': y2-y1},
                        'region_attributes': {'name': 'not_defined',
                                              'image_quality': {'good': True,
                                                                'frontal': True,
                                                                'good_illumination': True},
                                              'class': {typ: True}}}
                )
            else:
                Main_data[file_key] = {'filename': f_name,
                                       'size': os.path.getsize(save_dir+'/'+f_name),
                                       'regions': [{'shape_attributes': {'name': 'rect',
                                                                         'x': x1,
                                                                         'y': y1,
                                                                         'width': x2-x1,
                                                                         'height': y2-y1},
                                                   'region_attributes': {'name': 'not_defined',
                                                                         'image_quality': {'good': True,
                                                                                           'frontal': True,
                                                                                           'good_illumination': True},
                                                                         'class': {typ: True}}}],
                                       'file_attributes': {'caption': '', 'public_domain': 'no', 'image_url': ''}}
    # print(Main_data[file_key])
    # print('=====================================')


In [420]:
images=get_images()
aug_images=[]
points=[]
multiple_points=[]
for i in data:
    if i.split(':')[0] == 'filename':
        if len(multiple_points)!=0:
            augment_img(filename,image,multiple_points)
        multiple_points=[]
        image=images[i.split(':')[1]]
        filename=i.split(':')[1].split('.')[0]
    else:
        if i.split(":")[0] in ['x','y','width','height']:
            points.append(i.split(":")[1])
        else:
            x,y,w,h=map(int,points)
            cls=int(i.split(':')[0])
            temp=[x,y,x+w,y+h,cls]
            multiple_points.append(temp)
            points=temp=[]
if len(multiple_points)!=0:
        augment_img(filename,image,multiple_points)

f=json.dumps(Main_data,indent=4)
with open(os.path.join('new_data.json'),'w') as fil:
    fil.write(f)